# Conversational agent

In [6]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain.tools import tool

In [3]:
import requests
from pydantic.v1 import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [4]:
import wikipedia

@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [5]:
tools = [get_current_temperature, search_wikipedia]

In [7]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [8]:
functions = [convert_to_openai_function(f) for f in tools]
model = ChatOpenAI(temperature=0, model='gpt-4o-mini').bind(functions=functions)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [10]:
result = chain.invoke({"input": "what is the weather is sf?"})
result

AgentActionMessageLog(tool='get_current_temperature', tool_input={'latitude': 37.7749, 'longitude': -122.4194}, log="\nInvoking: `get_current_temperature` with `{'latitude': 37.7749, 'longitude': -122.4194}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 108, 'total_tokens': 134}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_661538dc1f', 'finish_reason': 'function_call', 'logprobs': None}, id='run-81c81bce-065d-4bb2-9f19-5ee37d78b91b-0', usage_metadata={'input_tokens': 108, 'output_tokens': 26, 'total_tokens': 134})])

In [11]:
result.tool

'get_current_temperature'

In [12]:
result.tool_input

{'latitude': 37.7749, 'longitude': -122.4194}

In [13]:
from langchain.prompts import MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [14]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [15]:
result1 = chain.invoke({
    "input": "what is the weather is sf?",
    "agent_scratchpad": []
})

In [16]:
result1.tool

'get_current_temperature'

In [18]:
observation = get_current_temperature.invoke(result1.tool_input)

/var/folders/zy/mbtr1z_97_x349r8bdfs7l100000gn/T/ipykernel_63824/3697834985.py:32: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc_time = datetime.datetime.utcnow()


In [19]:
observation

'The current temperature is 22.2°C'

In [20]:
type(result1)

langchain_core.agents.AgentActionMessageLog

In [21]:
from langchain.agents.format_scratchpad import format_to_openai_functions

In [22]:
result1.message_log

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 108, 'total_tokens': 134}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_661538dc1f', 'finish_reason': 'function_call', 'logprobs': None}, id='run-094b6177-04bd-48bc-b413-bf867b4ae9b7-0', usage_metadata={'input_tokens': 108, 'output_tokens': 26, 'total_tokens': 134})]

In [23]:
format_to_openai_functions([(result1, observation), ])

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 108, 'total_tokens': 134}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_661538dc1f', 'finish_reason': 'function_call', 'logprobs': None}, id='run-094b6177-04bd-48bc-b413-bf867b4ae9b7-0', usage_metadata={'input_tokens': 108, 'output_tokens': 26, 'total_tokens': 134}),
 FunctionMessage(content='The current temperature is 22.2°C', name='get_current_temperature')]

In [24]:
result2 = chain.invoke({
    "input": "what is the weather is sf?", 
    "agent_scratchpad": format_to_openai_functions([(result1, observation)])
})

In [25]:
result2

AgentFinish(return_values={'output': 'The weather in San Francisco is a lovely 22.2°C. Not too shabby, right? Perfect for a stroll or maybe a little outdoor brunch! 🌞'}, log='The weather in San Francisco is a lovely 22.2°C. Not too shabby, right? Perfect for a stroll or maybe a little outdoor brunch! 🌞')

In [26]:
from langchain.schema.agent import AgentFinish
def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = chain.invoke({
            "input": user_input, 
            "agent_scratchpad": format_to_openai_functions(intermediate_steps)
        })
        if isinstance(result, AgentFinish):
            return result
        tool = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result, observation))

In [27]:
from langchain.schema.runnable import RunnablePassthrough
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | chain

In [28]:
def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = agent_chain.invoke({
            "input": user_input, 
            "intermediate_steps": intermediate_steps
        })
        if isinstance(result, AgentFinish):
            return result
        tool = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result, observation))

In [29]:
run_agent("what is the weather is sf?")

/var/folders/zy/mbtr1z_97_x349r8bdfs7l100000gn/T/ipykernel_63824/3697834985.py:32: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc_time = datetime.datetime.utcnow()


AgentFinish(return_values={'output': 'The weather in San Francisco is a lovely 22.2°C. Not too shabby, right? Perfect for a stroll or maybe a little outdoor brunch! 🌞'}, log='The weather in San Francisco is a lovely 22.2°C. Not too shabby, right? Perfect for a stroll or maybe a little outdoor brunch! 🌞')

In [30]:
run_agent("what is langchain?")

AgentFinish(return_values={'output': "LangChain is a framework that makes it easier to create applications using large language models (LLMs). Think of it as a toolkit for building things like chatbots, document analysis tools, and code analyzers. It was launched in October 2022 by Harrison Chase and quickly gained traction, attracting contributions from hundreds of developers and even securing over $20 million in funding by April 2023.\n\nIn 2023, they introduced the LangChain Expression Language (LCEL) for defining action chains and LangServe, a tool for deploying LCEL code as a production-ready API. LangChain supports a wide range of integrations, from cloud storage services to various databases and even web scraping tools. So, if you're into AI and want to whip up some cool applications, LangChain might just be your new best friend!"}, log="LangChain is a framework that makes it easier to create applications using large language models (LLMs). Think of it as a toolkit for building 

In [31]:
run_agent("hi!")

AgentFinish(return_values={'output': "Hey there! What's up? Need some help or just here to chat?"}, log="Hey there! What's up? Need some help or just here to chat?")

In [32]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True)

In [33]:
agent_executor.invoke({"input": "what is langchain?"})



> Entering new AgentExecutor chain...

Invoking: `search_wikipedia` with `{'query': 'LangChain'}`


Page: LangChain
Summary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.

LangChain was launched in October 2022 as an open source project by Harrison Chase, while working at machine learning startup Robust Intelligence. The project quickly garnered popularity, with improvements from hundreds of contributors on GitHub, trending discussions on Twitter, lively activity on the project's Discord server, many YouTube tutorials, and meetups in San Francisco and London. In April 2023, LangChain had incorporated and the new startup raised over $20 million in funding at a valuation of at least $200 million from venture firm Sequoia Capit

{'input': 'what is langchain?',
 'output': "LangChain is a framework that makes it easier to create applications using large language models (LLMs). Think of it as a toolkit for building things like chatbots, document analysis tools, and code analyzers. It was launched in October 2022 by Harrison Chase and quickly gained traction, thanks to contributions from a ton of developers and a vibrant community.\n\nBy April 2023, LangChain had become a startup and raised over $20 million in funding. It introduced some cool features like the LangChain Expression Language (LCEL) for defining action chains and LangServe, a tool for deploying code as a production-ready API.\n\nLangChain can integrate with various systems, including cloud storage from Amazon, Google, and Microsoft, and it supports a wide range of functionalities, from summarizing documents to generating synthetic data. So, if you're into AI and want to build something nifty, LangChain might just be your new best friend!"}

In [34]:
agent_executor.invoke({"input": "my name is bob"})



> Entering new AgentExecutor chain...
Well, hello there, Bob! What can I do for you today?

> Finished chain.


{'input': 'my name is bob',
 'output': 'Well, hello there, Bob! What can I do for you today?'}

In [35]:
agent_executor.invoke({"input": "what is my name"})



> Entering new AgentExecutor chain...
Well, I don't have a crystal ball, so I can't just guess your name. But if you tell me, I promise to remember it for our chat!

> Finished chain.


{'input': 'what is my name',
 'output': "Well, I don't have a crystal ball, so I can't just guess your name. But if you tell me, I promise to remember it for our chat!"}

In [36]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [37]:
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | prompt | model | OpenAIFunctionsAgentOutputParser()

In [38]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")

In [39]:
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True, memory=memory)

In [40]:
agent_executor.invoke({"input": "my name is bob"})



> Entering new AgentExecutor chain...
Well, hello there, Bob! What can I do for you today?

> Finished chain.


{'input': 'my name is bob',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Well, hello there, Bob! What can I do for you today?')],
 'output': 'Well, hello there, Bob! What can I do for you today?'}

In [41]:
agent_executor.invoke({"input": "whats my name"})



> Entering new AgentExecutor chain...
Your name is Bob! Got any other questions, or are we just playing a game of "Guess Who"?

> Finished chain.


{'input': 'whats my name',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Well, hello there, Bob! What can I do for you today?'),
  HumanMessage(content='whats my name'),
  AIMessage(content='Your name is Bob! Got any other questions, or are we just playing a game of "Guess Who"?')],
 'output': 'Your name is Bob! Got any other questions, or are we just playing a game of "Guess Who"?'}

In [42]:
agent_executor.invoke({"input": "whats the weather in sf?"})



> Entering new AgentExecutor chain...

Invoking: `get_current_temperature` with `{'latitude': 37.7749, 'longitude': -122.4194}`


The current temperature is 22.2°C

/var/folders/zy/mbtr1z_97_x349r8bdfs7l100000gn/T/ipykernel_63824/3697834985.py:32: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc_time = datetime.datetime.utcnow()


The weather in San Francisco is a lovely 22.2°C. Perfect for a stroll or, you know, just pretending to enjoy the outdoors! Need anything else?

> Finished chain.


{'input': 'whats the weather in sf?',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Well, hello there, Bob! What can I do for you today?'),
  HumanMessage(content='whats my name'),
  AIMessage(content='Your name is Bob! Got any other questions, or are we just playing a game of "Guess Who"?'),
  HumanMessage(content='whats the weather in sf?'),
  AIMessage(content='The weather in San Francisco is a lovely 22.2°C. Perfect for a stroll or, you know, just pretending to enjoy the outdoors! Need anything else?')],
 'output': 'The weather in San Francisco is a lovely 22.2°C. Perfect for a stroll or, you know, just pretending to enjoy the outdoors! Need anything else?'}

### Create a chatbot

In [43]:
@tool
def create_your_own(query: str) -> str:
    """This function can do whatever you would like once you fill it in """
    print(type(query))
    return query[::-1]

In [44]:
tools = [get_current_temperature, search_wikipedia, create_your_own]

In [ ]:
import panel as pn  # GUI
pn.extension()
import panel as pn
import param

class cbfs(param.Parameterized):
    
    def __init__(self, tools, **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.functions = [format_tool_to_openai_function(f) for f in tools]
        self.model = ChatOpenAI(temperature=0).bind(functions=self.functions)
        self.memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are helpful but sassy assistant"),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])
        self.chain = RunnablePassthrough.assign(
            agent_scratchpad = lambda x: format_to_openai_functions(x["intermediate_steps"])
        ) | self.prompt | self.model | OpenAIFunctionsAgentOutputParser()
        self.qa = AgentExecutor(agent=self.chain, tools=tools, verbose=False, memory=self.memory)
    
    def convchain(self, query):
        if not query:
            return
        inp.value = ''
        result = self.qa.invoke({"input": query})
        self.answer = result['output'] 
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=450)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=450, styles={'background-color': '#F6F6F6'}))
        ])
        return pn.WidgetBox(*self.panels, scroll=True)


    def clr_history(self,count=0):
        self.chat_history = []
        return 

In [ ]:
cb = cbfs(tools)

inp = pn.widgets.TextInput( placeholder='Enter text here…')

conversation = pn.bind(cb.convchain, inp) 

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=400),
    pn.layout.Divider(),
)

dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# QnA_Bot')),
    pn.Tabs(('Conversation', tab1))
)
dashboard